Following the tutorial for [how to design a part in build123d](https://build123d.readthedocs.io/en/stable/tutorial_design.html)

In [5]:
from build123d import (
    BuildLine,
    BuildSketch,
    FilletPolyline,
    MM,
    Plane,
    Side,
    make_face,
    mirror,
    offset,
)
from ocp_vscode import show
height = 40 * MM
length = 60 * MM
thickness = 10 * MM
bend_radius = 5 * MM

In [6]:
with BuildSketch() as sketch:
    with BuildLine() as profile:
        FilletPolyline(
            (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
        )
        offset(amount=thickness, side=Side.LEFT)
    make_face()
    mirror(about=Plane.YZ)

show(sketch)

RuntimeError: Cannot access viewer config. Is the viewer running?
("Cannot access viewer status. Is the viewer running?\n('none',)",)